# Clip Model



## Clip with lighting module

### Packages Installed and Imports

#### installations

In [1]:
%%capture
!pip install cairosvg
!pip install ftfy regex tqdm
!pip install transformers
!pip install pytorch_lightning

#### Import modules

In [2]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import os
import shutil
import zipfile
import random
from zipfile import ZipFile
import torch.nn.functional as F
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel, AdamW
import torch.utils.data as data
from torch.utils.data import Subset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
import json
from PIL import Image
import torchvision.transforms as transforms
from transformers.models.clip.modeling_clip import CLIPOutput
from pytorch_lightning.callbacks import ModelCheckpoint

import tensorflow as tf
import tensorflow_datasets as tfds

import io
import cairosvg

### DataSet

#### Dataset AWS

In [3]:
class ClipSWDataset(data.Dataset):
    def __init__(self, dataset,processor):
        self.processor = processor
        self.dataset=[]
        self.filter_data(dataset)

    def create_new_datum(self,datum,term,correct_image):
        new_datum = datum.copy()
        if self.processor(term,return_tensors="pt").input_ids.to(device).shape[1] < 77:
           new_datum['terms'] = ["\n ",term]
           new_datum['sign_writing_images'] = [correct_image]
        
        return new_datum
    
    def filter_datum(self,datum):
        return len(datum['terms']) > 1 and len(datum['sign_writing_images']) > 0 and datum['assumed_spoken_language_code'] == 'en'
            
    def filter_data(self,dataset):
        self.dataset = []

        for datum in dataset:
            if self.filter_datum(datum):
                # Got the image where there is just one image that describe the sign writing
                correct_image = None
                for images in datum['sign_writing_images']:
                    if len(images) == 1:
                        correct_image = images
                        break
                # Check it there is one image that describe the text
                if correct_image:
                    for term in datum['terms'][1:]:
                        if len(term) < 77:
                            new_datum = self.create_new_datum(datum,term,correct_image)
                            self.dataset.append(new_datum)
                        
    def __len__(self): 
        return len(self.dataset)

    def __getitem__(self, index):
        # Change it , use processor
        preprocess_transforms = transforms.Compose([
            transforms.Lambda(lambda rgba_img: rgba_img.convert('RGB')),
            transforms.Resize((224, 224)),
            transforms.ToTensor()])
        svg_file_path = self.dataset[index]['sign_writing_images'][0][0]
        png_bytes = cairosvg.svg2png(url=svg_file_path)
        curr_img = Image.open(io.BytesIO(png_bytes))  
        image_tensor = preprocess_transforms(curr_img)
        text = self.dataset[index]["terms"][1]
        return image_tensor, text

#### Split data

In [4]:
def split_into_train_and_test():
    json_folder_path = 'signwriting-images/dataset-outputs-new'
    json_files = []
    for dirpath, _, filenames in os.walk(json_folder_path):
        for filename in filenames:
            if filename.endswith('.json'):
              json_file_path = os.path.join(dirpath, filename)
              # Open the JSON file for reading
              with open(json_file_path, 'r') as f:
                  json_str = f.read()
                  # Parse the JSON string into a Python object
                  data = json.loads(json_str)
              json_files.append(data)
    
    train_size = int(0.8 * len(json_files))
    test_size = len(json_files) - train_size
    train_dataset, test_dataset = random_split(json_files, [train_size, test_size])

    return train_dataset, test_dataset

### Loss

In [5]:
# Define the contrastive loss function
def contrastive_loss(image_rep, text_rep):
    bs = image_rep.size(0)
    device = image_rep.device
    similarity_matrix = torch.matmul(image_rep, text_rep.T)
    mask = torch.eye(bs, device=device).bool()
    sim_pos = torch.diagonal(similarity_matrix)
    sim_neg = similarity_matrix[~mask].view(bs, -1).max(dim=1).values
    loss = (-torch.log(sim_pos / (sim_pos + sim_neg))).mean()
    return loss

### Model

In [6]:
class CLIPTrainer(pl.LightningModule):
    def __init__(self, model_name_or_path, loss_fn,learning_rate=3e-5,batch_size=64):
        super().__init__()
        self.processor  = CLIPProcessor.from_pretrained(model_name_or_path)
        self.model: CLIPModel = CLIPModel.from_pretrained(model_name_or_path)
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.loss_fn = loss_fn
    
    def forward(self, input_ids, attention_mask, image):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, visual_inputs=image)

    def pad_text_tokenized(self, texts):
        texts = [self.processor(text, return_tensors="pt").input_ids.to(device) for text in texts]

        # Pad tensors with zeros to make them the same size
        padded_tensors = []
        max_len = max([t.shape[1] for t in texts])
        for t in texts:
            padded_tensor = F.pad(t, (0, max_len - t.shape[1]), mode='constant', value=0)
            padded_tensors.append(padded_tensor)

        stacked_tensor = torch.stack(padded_tensors)
        stacked_tensor = torch.squeeze(stacked_tensor, dim=1)
        return stacked_tensor
    
    def training_step(self, batch, batch_idx):
        image, texts = batch
        stacked_tensor = self.pad_text_tokenized(texts)      

        outputs: CLIPOutput = self.model(stacked_tensor, image)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text
        loss = self.loss_fn(logits_per_image, logits_per_text)
        self.log('train_loss', loss, on_epoch=True, logger=True, prog_bar=True,batch_size=self.batch_size)
        return loss
    
    def validation_step(self, batch, batch_idx):
        image, texts = batch
        stacked_tensor = self.pad_text_tokenized(texts)      

        outputs: CLIPOutput = self.model(stacked_tensor, image)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text
        loss = self.loss_fn(logits_per_image, logits_per_text)
        self.log('val_loss', loss, on_epoch=True, logger=True, prog_bar=True, batch_size=self.batch_size)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def prepare_data(self):
      train_dataset, test_dataset = split_into_train_and_test()
      print(f"Len Before train: {len(train_dataset)}")
      print(f"Len Before test: {len(test_dataset)}")
      self.train_dataset = ClipSWDataset(train_dataset, self.processor)
      self.test_dataset = ClipSWDataset(test_dataset, self.processor)
      print(f"Len After train: {len(self.train_dataset)}")
      print(f"Len After test: {len(self.test_dataset)}")

        
    def train_dataloader(self):
       return DataLoader(dataset=self.train_dataset,
                         batch_size=self.batch_size,
                         shuffle=True,
                         num_workers=2)
    
    def val_dataloader(self):
      return DataLoader(dataset=self.test_dataset,
                        batch_size=self.batch_size,
                        shuffle=False,
                        num_workers=2)

## Train Model

### Got zipfile

In [7]:
%%script echo skipping
SIGN_BANK_DATASET = "signwriting-images"

!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-41JbcJCtx8tdmnGq-krrMe-6aiqVSPo # signbank

with ZipFile(f'{SIGN_BANK_DATASET}.zip', 'r') as zObject:
  zObject.extractall(path=SIGN_BANK_DATASET)

skipping


### Training

In [8]:
%%script echo skipping
from google.colab import drive
drive.mount('/content/drive')

checkpoint_path = "drive/MyDrive/sign_writing_trainer/"
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_path, filename="sw-model",save_top_k=1, monitor='val_loss')
callbacks = [checkpoint_callback]
model = CLIPTrainer('openai/clip-vit-base-patch32',contrastive_loss, learning_rate=1e-4,batch_size=100)
device = "cuda" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator='gpu', max_epochs=10,callbacks=callbacks)
trainer.fit(model)

skipping


## Load Model & Create vectors

### Got zipfile

In [9]:
%%script echo skipping
SIGN_2_MINT_SVGS_DATASET = "sign2mint-Svgs"

!pip install -U --no-cache-dir gdown --pre
!gdown --id 1ZXDsSKodxbHGLTUWQRjBTc27KcUmnoYs # model
!gdown --id 1vmakUlaApbEtVz6rq9M-1pN55aGPDfHc # sign2mint jsonl
!gdown --id 1kryyoTYEqMRTdUCuODU1rd1fBQo7C4mY # sign2mint svgs

with ZipFile(f'{SIGN_2_MINT_SVGS_DATASET}.zip', 'r') as zObject:
  zObject.extractall(path=SIGN_2_MINT_SVGS_DATASET)

skipping


### Sign writing loaded model

In [10]:
class SignWritingModel:
    def __init__(self, model_name_or_path: str, checkpoint_path: str):
        args = dict(
            model_name_or_path=model_name_or_path,
            loss_fn=contrastive_loss
        )
        self.trainer: CLIPTrainer = CLIPTrainer.load_from_checkpoint(
            checkpoint_path=checkpoint_path, **args)
        self.similarity_model: CLIPModel = self.trainer.model.to(device)
        self.processor = self.trainer.processor

    def sign_writing_signature(self, text: str, image) -> torch.Tensor:
        inputs = self.trainer.processor(text=text,images=image,return_tensors="pt").to(device)
        encoded_text = self.similarity_model.get_text_features(input_ids=inputs.input_ids)
        encoded_image = self.similarity_model.get_image_features(pixel_values=inputs.pixel_values)
        encoded_vector = torch.cat((encoded_image, encoded_text), dim=-1)
        return encoded_vector

### Create sign2mint vectors

In [11]:
def store_sign2mint_vectors(model: SignWritingModel, dir_images_path: str):
    with open('sign2mint.jsonl', 'r') as f:
        with open("sign2mint-Vectors.jsonl", "w") as s:
            sign2mint = [json.loads(s) for s in list(f)]
            # vectors_list = []
            for row in tqdm(sign2mint):
                uid = row["doc"]["uid"]
                word = row["captions"][0]["transcription"]
                png_data = cairosvg.svg2png(url=f'{dir_images_path}/{uid}.svg', write_to=None)
                curr_img = Image.open(io.BytesIO(png_data))

                embedding_vector = model.sign_writing_signature(
                    text=word, image=curr_img)
                result = {"word": word, "uid": uid, "embedding_vector": embedding_vector.tolist()}
                json_string = json.dumps(result)
                s.write(json_string + "\n")

In [12]:
%%script echo skipping
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SignWritingModel('openai/clip-vit-base-patch32', 'sw_model.ckpt')
store_sign2mint_vectors(model=model, dir_images_path=SIGN_2_MINT_SVGS_DATASET)

skipping


# Spoken to text

#### installations

In [13]:
%%capture
!pip install SpeechRecognition

## Import modules

In [14]:
import speech_recognition as sr

## Generate text from spoken

In [15]:
class AudioRecorder:
    def __init__(self):
        self.recognizer = sr.Recognizer()

    def convert_to_text(self, wav_filename: str):
        with sr.AudioFile(wav_filename) as source:
            audio_text = self.recognizer.record(source)

        try:
            s = self.recognizer.recognize_google(audio_text)
            print("Path: ", wav_filename, " Text: " + s)
        except Exception as e:
            s = "wave_path: " + wav_filename + " Exception: " + str(e)
            print(s)
        return s

# Generate video from text

### Got zipfile

In [16]:
FONT_DB = "font_db"
SIGN_2_MINT_VECTORS_DATASET = "sign2mint-Vectors"
SIGN_2_MINT_VIDEOS_DATASET = "sign2mint-Videos"

!pip install -U --no-cache-dir gdown --pre
!gdown --id 1Tdry97vVyXUL-aFdQ97QDElw_lb4smKL # font_db
!gdown --id 1_efH5P-lfg6coxkOmupOoutGVzaKgRPr # sign2mint vector
!gdown --id 1cpmCWgpRGZeC2GMfdz2ROrrhpn6sp3Bo # sign2mint videos

with ZipFile(f'{FONT_DB}.zip', 'r') as zObject:
  zObject.extractall(path=FONT_DB)

subprocess.call('npm install', cwd='font_db/font_db', shell=True)

with ZipFile(f'{SIGN_2_MINT_VECTORS_DATASET}.zip', 'r') as zObject:
  zObject.extractall(path=SIGN_2_MINT_VECTORS_DATASET.split('-')[1])

with ZipFile(f'{SIGN_2_MINT_VIDEOS_DATASET}.zip', 'r') as zObject:
  zObject.extractall(path=SIGN_2_MINT_VIDEOS_DATASET.split('-')[1])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Tdry97vVyXUL-aFdQ97QDElw_lb4smKL
To: /content/font_db.zip
100% 15.2M/15.2M [00:00<00:00, 24.2MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1_efH5P-lfg6coxkOmupOoutGVzaKgRPr
From (redirected): https://drive.google.com/uc?id=1_efH5P-lfg6coxkOmupOo

## Import modules

In [17]:
import json
import os
import shutil
from torch import Tensor
from PIL import Image
import torch
import subprocess
import cairosvg
import io
import numpy as np
from moviepy.video.compositing.concatenate import concatenate_videoclips
from moviepy.video.io.VideoFileClip import VideoFileClip

## Utils

### sign writing util

In [18]:
def api_call_spoken2sign(payload):
    url = 'https://pub.cl.uzh.ch/demo/signwriting/spoken2sign'
    response = requests.post(url, json=payload).json()
    return response['translations'][0]

### Video util

In [19]:
def concate_two_videos(first_video_name: str,
                       second_video_name: str,
                       final_video_name: str):
    # Load the two video files
    first_video = VideoFileClip(f'{first_video_name}.mp4')
    second_video = VideoFileClip(f'{second_video_name}.mp4')

    # Concatenate the clips
    final_video = concatenate_videoclips([first_video, second_video])

    # Write the final clip to a new file
    final_video.write_videofile(f'{final_video_name}.mp4')

### Similarity util

In [20]:
def diff(a, b, similar=False):
    sim = ""
    if similar is not True:
        sim = "Not"
    dot_product = np.dot(a, b)

    # Calculate the magnitudes
    magnitude_a = np.linalg.norm(a)
    magnitude_b = np.linalg.norm(b)

    # Calculate the cosine similarity
    cosine_similarity = dot_product / (magnitude_a * magnitude_b)

    return cosine_similarity


def check_cosin(traget_vector: Tensor, data) -> list:
    differences = []
    for dt in data:
        # dt = { "text", "image_text_encoded", "pose_url"}
        vec = np.array(dt['embedding_vector']).flatten()
        img = np.array(traget_vector.tolist()).flatten()
        cos_diff = diff(vec, img)
        differences.append((dt['word'], dt['uid'], cos_diff))

    differences = sorted(differences, key=lambda diff_images: diff_images[2],reverse=True)

    return differences

## Get video

In [73]:
class FlowManager:
    def __init__(self, model: SignWritingModel,
                 encoded_vectors_path: str):
        self.model = model

        with open(encoded_vectors_path,'r') as f:
            self.data = [json.loads(s) for s in list(f)]

    def generate_image(self, spoken_word: str) -> Image:
        encoded_sw = api_call_spoken2sign({
            "country_code": 'us',
            "language_code": 'en',
            "text": spoken_word,
            "translation_type": "sent"
        })
        output = subprocess.check_output(
            f'node fsw/fsw-sign-svg {encoded_sw}', cwd='font_db/font_db', shell=True)
        output_str = output.decode('utf-8')
        png_bytes = cairosvg.svg2png(output_str)
        curr_img = Image.open(io.BytesIO(png_bytes))

        return curr_img

    def get_word_uid(self, word: str, data):
        for dt in data:
            if dt['word'] == word:
                return dt["uid"]

        return None

    def run(self, sentence: str):
        words_sentence: list[str] = sentence.replace('`', '').split(' ')

        uid_sentence = []
        for text in words_sentence:
            word_uid = self.get_word_uid(text, self.data)
            if not word_uid:
                image: Image = self.generate_image(spoken_word=text)
                encoded_vector: torch.Tensor = self.model.sign_writing_signature(
                    text=text, image=image)
                similar_tensor_images = check_cosin(traget_vector=encoded_vector,
                                                    data=self.data)
                word_uid = similar_tensor_images[0][1]

            uid_sentence.append(f'{SIGN_2_MINT_VIDEOS_DATASET}/{word_uid}.mp4')

        final_video_name = 'final_video.mp4'
        video_files_name: list = uid_sentence
        if len(video_files_name) > 0:
            shutil.copy(video_files_name[0], final_video_name)
        for video_file_name in video_files_name[1:]:
            concate_two_videos(
                final_video_name, video_file_name, final_video_name)

# Telegram Bot

#### installations

In [22]:
!apt install ffmpeg
!pip install fastapi
!pip install uvicorn
!pip install python-telegram-bot
!pip install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.4/545.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.9 MB/s eta 0:00:00
   

### Got files

In [23]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1ZXDsSKodxbHGLTUWQRjBTc27KcUmnoYs # model
!gdown --id 1vmakUlaApbEtVz6rq9M-1pN55aGPDfHc # sign2mint jsonl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ZXDsSKodxbHGLTUWQRjBTc27KcUmnoYs
From (redirected): https://drive.google.com/uc?id=1ZXDsSKodxbHGLTUWQRjBTc27KcUmnoYs&confirm=t&uuid=6a5164c4-07d0-487b-b42d-45ce58c4444e
To: /content/sw_model.ckpt
100% 1.82G/1.82G [00:14<00:00, 122MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1vmakUlaApbEtVz6rq9M-1pN55aGPDfHc
To: /content/sign2mint.jsonl
100% 5.30M/5.30M [00:00<00:00, 190MB/s]


## Import modules

In [24]:
import os.path
import subprocess
import requests
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes,filters
from pydub import AudioSegment

## Global Variables

In [75]:
TOKEN = '5993731273:AAHnf_LvkKJO-oWdXf_O20eDvRXKBF7Bkco'
FFMPEG_PATH = '/usr/bin/ffmpeg'
os.environ["ffmpeg"] = FFMPEG_PATH

AudioSegment.converter = FFMPEG_PATH
AudioSegment.ffmpeg = FFMPEG_PATH
device = "cuda" if torch.cuda.is_available() else "cpu"
flow_manager = FlowManager(model=SignWritingModel(
    model_name_or_path='openai/clip-vit-base-patch32',
    checkpoint_path='sw_model.ckpt'),
    encoded_vectors_path='sign2mint-vectors/sign2mint-vectors.jsonl')

## Commands

In [28]:
# TODO: fix audio shit, create api for the model(easy), queues ?
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
    )

In [54]:
async def handle_voice_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    file_name = 'audio'
    voice = update.message.voice
    file = await voice.get_file()

    response = requests.get(file.file_path)

    with open(f'{file_name}.oga', 'wb') as f:
        f.write(response.content)

    if os.path.exists(f'{file_name}.wav'):
        os.remove(f'{file_name}.wav')

    subprocess.run(['ffmpeg', '-i', f'{file_name}.oga', f'{file_name}.wav'])

    recorder = AudioRecorder()
    text = recorder.convert_to_text(wav_filename=f'{file_name}.wav')
    await update.message.reply_text(text)

    if not text.__contains__('Exception'):
        flow_manager.run(text)
        await context.bot.send_video(chat_id=update.effective_chat.id,
                                     video=open('final_video.mp4', 'rb'))
        os.remove('final_video.mp4')

    os.remove(f'{file_name}.oga')
    os.remove(f'{file_name}.wav')

In [79]:
async def handle_text_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    text = update.message.text
    flow_manager.run(text)
    await context.bot.send_video(chat_id=update.effective_chat.id,
                                  video=open('final_video.mp4', 'rb'))
    os.remove('final_video.mp4')

## Start Bot

In [47]:
import nest_asyncio
nest_asyncio.apply()

In [80]:
application = Application.builder().token(TOKEN).build()
application.add_handler(CommandHandler("start", start))
application.add_handler(MessageHandler(filters.TEXT, handle_text_message))
application.add_handler(MessageHandler(filters.VOICE, handle_voice_message))

print("Running application...")
application.run_polling()

Running application...


RuntimeError: ignored